In [1]:
#!pip install gensim

In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
import cudf
import glob

from gensim.test.utils import common_texts
from gensim.models import Word2Vec

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [4]:
import pickle

igfold = 0
sessions = pickle.load(open('./data/sessions_eval.pickle', 'rb'))
if igfold == 0:
    sess_eval = sessions[0]+sessions[1]
elif igfold == 1:
    sess_eval = sessions[2]+sessions[3]
elif igfold == 2:
    sess_eval = sessions[4]+sessions[5]
elif igfold == 3:
    sess_eval = sessions[6]+sessions[7]
elif igfold == 4:
    sess_eval = sessions[8]+sessions[9]
print(len(sess_eval))

356748


In [5]:
files_aid = sorted(glob.glob('../../data/' + '/train/interim/*.parquet')) + glob.glob('../../data/test.parquet') + glob.glob('./data/xgb_train_x.parquet')

out = []
for file in files_aid:
    df_tmp = cudf.read_parquet(file)
    if 'xgb_train_x.parquet' in file:
        print(file)
        df_tmp = df_tmp[df_tmp['session'].isin(sess_eval)]
    else:
        df_tmp = df_tmp[~(df_tmp['session'].isin(sess_eval))]
    out.append(df_tmp)
df = cudf.concat(out)

./data/xgb_train_x.parquet


In [6]:
df = df.sort_values(['session', 'ts'])

In [7]:
df = df[['session', 'aid']].groupby(['session']).agg(list)

In [8]:
sentences = df['aid'].to_pandas().to_list()

In [9]:
%%time

sentences = [x.tolist() for x in sentences]

CPU times: user 1min 10s, sys: 32.9 s, total: 1min 43s
Wall time: 1min 43s


In [10]:
%%time

w2vec = Word2Vec(
    sentences=sentences, 
    vector_size=50, 
    epochs=5, 
    sg=1, 
    window=3, 
    sample=1e-3, 
    ns_exponent=1, 
    min_count=1, 
    workers=8
)

CPU times: user 3h 20min 43s, sys: 1min 3s, total: 3h 21min 47s
Wall time: 31min 34s


In [11]:
aid2idx = {aid: i for i, aid in enumerate(w2vec.wv.index_to_key)}

In [12]:
emb = {}
for aid, idx in aid2idx.items():
    emb[aid] = w2vec.wv.vectors[idx]

In [13]:
pickle.dump(emb, open('./data_folds/fold_' + str(igfold) + '/word2vec.emb', 'wb'))